<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Copy_of_Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read dev dataset**

In [ ]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

**Read train dataset**

In [ ]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

# **Tokenize Data**

**install packages**

In [ ]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

     |████████████████████████████████| 432 kB 5.2 MB/s 
     |████████████████████████████████| 174 kB 31.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=6d5c6c0a624737ebfe540d7a55f32c3bbe8cab15cecc1390d2db46787eca7c26
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
# Import the package
import stanza

In [ ]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-28 16:48:11 INFO: Downloading default packages for language: fa (Persian)...


2022-01-28 16:48:19 INFO: Finished downloading models and saved to /root/stanza_resources.


**create models**

In [ ]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-28 16:48:22 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-28 16:48:22 INFO: Use device: gpu
2022-01-28 16:48:22 INFO: Loading: tokenize


Building a Persian pipeline...


2022-01-28 16:48:35 INFO: Loading: mwt
2022-01-28 16:48:35 INFO: Loading: pos
2022-01-28 16:48:35 INFO: Loading: lemma
2022-01-28 16:48:35 INFO: Loading: depparse
2022-01-28 16:48:36 INFO: Done loading processors!


**find tokens of dataframe and save in array**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)[:52465]):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


**save tokens array to csv file**

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

**convert tokens array to dataframe **

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

**save dataframe to csv file**

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

# **Read tokenized dataet**

**read from array csv file **

In [ ]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

# **Preproccessing**

**Remove punctuation**

In [ ]:
from types import new_class
# from string import punctuation
import re

punctuation=['!','"','#','$','%','&',"'",'(',')','*','+','،','-','.',','"/",':',';','<','=','>','?','@','[',"\\","]","^",'_','`','{','|','}','~']
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i[0].split(','):
    if j not in punctuation:
      sent.append(j)
  tokens_arr.append(sent)

**Analyze Reviews Length**

In [ ]:
# import matplotlib.pyplot as plt
reviews_len = [len(x) for x in tokens_arr]
reviews_len=np.array(reviews_len)

In [ ]:
# pd.Series(tokens_arr).hist()
# plt.show()
# pd.Series(tokens_arr).describe()

In [ ]:
print("min:",reviews_len.min(),"max:",reviews_len.max(),"min:",reviews_len.mean(),"len:",len(reviews_len))

min: 3 max: 362 min: 19.003015873015872 len: 56700


In [ ]:
print(np.count_nonzero(reviews_len > 80))

538


**Remove outliers**

In [ ]:
# cleaned_rokens_arr=[x for x in tokens_arr if len(x)<81]
# print(len(cleaned_rokens_arr))

ثریا 
اینجا کد ورد امبدنیگ رو میتونی ببینی 
کلمات مختلفی هم باهاش امتحان کردم اوکی شد

# **Word embedding**

In [ ]:
!pip install gensim
# !pip install nltk

In [ ]:
#main_code
import gensim
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/model.bin', binary=True)

In [ ]:
vector = model['خوب']
# see the shape of the vector (300,)
# vector.shape
list(vector)

In [ ]:
# model.most_similar('بهتر')

**calculate vacors for each word**

In [ ]:
vocabulary =model.wv.vocab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
print(len(vocabulary),type(vocabulary))

966446 <class 'dict'>


In [ ]:
vectors_arr=[]
for sent in tokens_arr:#cleaned_rokens_arr:
  vec_vocab=[list(model[word]) for word in sent if word in model.wv.vocab]# if model[word]
  vectors_arr.append(vec_vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
type(vectors_arr[0][0])

list

In [ ]:
from sys import getsizeof
getsizeof(vectors_arr)

457360

**save embedded words in csv file(too large!)>500mb**

In [ ]:
# #save vctors to array
# import csv
# with open('vectors_arr.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(vectors_arr)

In [ ]:
#to realease memory
tokens_arr=[]
tokens_array=[]

# **Prepare Train data for train**

## **Vectors**

**lenght of vectors**

In [ ]:
vectors_arr[:5]

In [ ]:
vectors_len = [len(x) for x in vectors_arr]
vectors=np.array(vectors_len)
max_vector_len=max(vectors_len)

In [ ]:
max_vector_len

351

In [ ]:
#to realease memory
vectors_len=[]

**Add padding to short comemnts**

In [ ]:
#add padding vetor 0
padding_vec=[0.0]*100
# padding_vec=np.array(padding_vec)
pad_embedded_array=[]
for vector in vectors_arr:
  if len(vector)<max_vector_len:
    padding=[padding_vec]*(max_vector_len-len(vector))
    vector=vector+padding
  pad_embedded_array.append(vector)

In [ ]:
pad_embedded_array[0]

## **Labels**

In [ ]:
l=train_df["label_id"]

In [ ]:
#to release memory
train_df=[]

# **Traingin LSTM**

**Import tensowerflow**

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
SEED = 20
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# from keras.layers import Embedding
from keras.layers import Input
# from keras.layers import TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras.models import Model
from keras.layers.core import Dense, Dropout,Activation
# import tqdm
# from tensorflow.keras import layers
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.python.keras.layers import Input
from keras.models import Sequential
# from my_classes import DataGenerator
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence

In [ ]:
# import pathlib
# import os
# np.set_printoptions(precision=4)

**Create Model**

In [ ]:
input_dim

100

In [ ]:
nb_samples = len(padding_vec)#55000
time_steps=max_vector_len#74
input_dim=100 #100
input_shape=(time_steps,input_dim)
n_epoch=1
batch_size=32

In [ ]:
# create the model
# embedding_weights = np.zeros((n_symbols, vocab_dim))
lstm_model = Sequential()
# model.add(LSTM(3NumberOfLSTM, return_sequences=True,input_shape=(YourSequenceLenght, YourWord2VecLenght)))
lstm_model.add(LSTM(batch_size,return_sequences=True, input_shape=input_shape))
# lstm_model.add(Dropout(0.2))
# lstm_model.add(Dense(40, activation='softmax'))
# lstm_model.add(Dense(20, activation='softmax'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.add(Activation('sigmoid'))
# lstm_model.add(Activation('softmax'))
print('Compiling the Model...')
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Compiling the Model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 351, 32)           17024     
                                                                 
 dense_2 (Dense)             (None, 351, 1)            33        
                                                                 
 activation_2 (Activation)   (None, 351, 1)            0         
                                                                 
Total params: 17,057
Trainable params: 17,057
Non-trainable params: 0
_________________________________________________________________


In [ ]:
traingen = CustomDataGen(pad_embedded_array,y_col=l_train,batch_size=batch_size, input_size=5)
# valgen = CustomDataGen(val_df,X_col={'path':'filename', 'bbox': 'region_shape_attributes'},
                      #  y_col={'name': 'name', 'type': 'type'},batch_size=batch_siz, input_size=target_size)

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, y_col,
                 batch_size,
                 input_size,
                 shuffle=True):
        
        self.df = df#.copy()
        # self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        # self.input_size = input_size
        self.shuffle = shuffle
        self.n = len(self.df)
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        
        # batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        # X, y = self.__get_data(batches)  
        print(type(self.y_col[index][0] ),type(self.df.iloc[index]))
        X,y=self.df[index],self.y_col.iloc[index]      
        print(x)
        print(y)
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size


In [ ]:
# train_df.fillna(value='', inplace=True)
lstm_model.fit(traingen,epochs=n_epoch,verbose=1)#,callbacks=[es, checkpoint]

In [ ]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((pad_embedded_array[:5], l_train[:5]))
for row in numeric_dataset.take(3):
  print(row)

**Train model**

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
checkpoint = ModelCheckpoint('best_model.h5', monitor='loss', mode='max', verbose=1, save_best_only=True)
x_train=pad_embedded_array
# tf.data.Dataset.from_tensors(pad_embedded_array)
y_train=l

In [ ]:
type(pad_embedded_array)

list

In [ ]:
for i in y_train[:10]:
  print(type(i),(i))

<class 'int'> 1
<class 'int'> 0
<class 'int'> 1
<class 'int'> 0
<class 'int'> 0
<class 'int'> 1
<class 'int'> 0
<class 'int'> 1
<class 'int'> 0
<class 'int'> 1


In [ ]:
type(x_train[0][0])

numpy.ndarray

In [ ]:
x_train_df=pd.DataFrame(x_train)

In [ ]:
type(x_train_df[:10])

pandas.core.frame.DataFrame

In [ ]:
type(pd.DataFrame(x_train)[:10])

pandas.core.frame.DataFrame

In [ ]:
(y_train.to_list()[:10])

[1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

In [ ]:
type(y_train[:10])

pandas.core.series.Series

In [ ]:
print( "Train..." )# batch_size=32
history =lstm_model.fit(pad_embedded_array[:10], y_train.to_list()[:10], batch_size=batch_size, epochs=n_epoch,verbose=1,callbacks=[es, checkpoint])
# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train...


ValueError: ignored

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

# **TEST**

**Prepare test data**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
test_tokenized_array=[]
for i in range(0,len(dev_df)):
  comment=dev_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  test_tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


In [ ]:
len(test_tokenized_array)

6300

In [ ]:
(test_tokenized_array[0])

['همه',
 'ساندویچ\u200cها',
 'سرد',
 'بود',
 'متاسفانه',
 'در',
 'صورتی',
 'که',
 'فاصله',
 'تا',
 'رستوران',
 'کمتر',
 'از',
 '۵',
 'دقیقه',
 'است',
 '.']

In [ ]:
# pd.DataFrame(test_tokenized_array).to_csv('testtokens_df.csv')
# #save tokens to array
# import csv
# with open('test_tokens_array.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(test_tokenized_arr)

In [ ]:
# #read test array csv file
# test_tokens_csv=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/testtokens_df.csv",header=None)#test_tokens_array.csv"
# test_tokens_arr=test_tokens_csv.to_numpy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
test_vectors_arr=[]
for sent in test_tokens_arr:
  vec_vocab=[model[word] for word in sent if word in vocabulary]
  test_vectors_arr.append(vec_vocab)

In [ ]:
# vectors_test_len = [len(x) for x in test_vectors_arr]
# vectors_test_len=np.array(vectors_test_len)
# max_vector_test_len=max(vectors_test_len)
# max_vector_test_len

79

In [ ]:
max_vector_test_len=80
#add padding vetor 0
padding_vec=[0.0]*100
padding_vec=np.array(padding_vec)
pad_vec_test_array=[]
for vector in test_vectors_arr:
  if len(vector)<max_vector_test_len:
    padding=[padding_vec]*(max_vector_test_len-len(vector))
    vector=vector+padding
  pad_vec_test_array.append(vector)

In [ ]:
x_test=pad_vec_test_array
y_test=dev_df["label_id"]

In [ ]:
# evaluate the model
scores = lstm_model.evaluate(x_test, y_test,batch_size, verbose=0)

Test Model

# **Prediction**

In [ ]:
Y_pred = model.predict(X_test)

# **Evaluation**

## **Confusion matrix**

In [ ]:
new_Y_pred = np.argmax(Y_pred, axis=1)
new_Y_test = np.argmax(Y_test, axis=1)

print(classification_report(new_Y_test, new_Y_pred))

mpl.style.use('seaborn')

conf_arr = np.zeros((num_classes, num_classes))

for i in range(len(new_Y_pred)):
        conf_arr[new_Y_pred[i]][new_Y_test[i]] += 1

print(conf_arr)

summ = conf_arr.sum()

df_cm = pd.DataFrame(conf_arr, 
  index = unique_cat1,
  columns = unique_cat1)

fig = plt.figure()

plt.clf()

ax = fig.add_subplot(111)
ax.set_aspect(1)

cmap = sb.cubehelix_palette(light=1, as_cmap=True)

res = sb.heatmap(df_cm, annot=True, vmin=0.0, vmax=np.max(conf_arr), fmt='.2f', cmap=cmap)

res.invert_yaxis()

plt.yticks([0.5,1.5,2.5,3.5,4.5,5.5], unique_cat1,va='center')

print('\n\n')

plt.title('Confusion Matrix')

plt.savefig('confusion_matrix.png', dpi=700, bbox_inches='tight' )

plt.show()

## **Accuracy**

In [ ]:
corrects = 0
for i in range(len(new_Y_pred)):
    if int(new_Y_pred[i]) is int(new_Y_test[i]):
        corrects += 1
        
accuracy = float(corrects / len(new_Y_pred))*100
print('Accuracy (using "{}" column): {} %'.format (column1, accuracy))

accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

## **Precision**

In [ ]:
#Similar to the binary case, we can define precision for each of the classes
precision = []
#we append the precision for each class to the variable 'precision'
for i in range(num_classes):
  precision.append(conf_arr[i][i] / np.sum(conf_arr[i], axis = 0))

print('Precisions per classes are : {}'.format(precision))

macro_precision = 100 * np.sum(precision) / num_classes

print('Macro averaged Precision is : {} %'.format(macro_precision))

weighted_precision = 0
total = 0
for i in range(num_classes):
  weighted_precision += np.sum(conf_arr, axis = 0)[i] * precision[i]
  total += np.sum(conf_arr, axis = 0)[i]

weighted_precision = 100 * weighted_precision / total
print('Weighted Precision is : {} %'.format(weighted_precision))

## **Recall**

In [ ]:
#Similar to the binary case, we can define recall for each of the classes
recall = []
#We append the recall for each class to the variable 'recall'
for i in range(num_classes):
  recall.append(conf_arr[i][i] / np.sum(conf_arr, axis = 0)[i])
  
print('Recalls per classes are : {}'.format(recall))

macro_recall = 100 * np.sum(recall) / num_classes
print('Macro averaged Recall is : {} %'.format(macro_recall))

weighted_recall = 0
for i in range(num_classes):
  weighted_recall += np.sum(conf_arr, axis = 0)[i] * recall[i]

weighted_recall = 100 * weighted_recall / total
print('Weighted Recall is : {} %'.format(weighted_recall))  

## **F1 score**

In [ ]:
#F1-score is a function of precision and recall
#we can now compute the per-class F1-score
f1_score = []
for i in range(num_classes):
  f1_score.append( 2*( (recall[i]*precision[i]) / (recall[i]+precision[i])))

print('F1-scores per classes are : {}'.format(f1_score))

macro_f1 = 100 * np.sum(f1_score) / num_classes

print('Macro averaged F1-score is : {} %'.format(macro_f1))

weighted_f1 = 0
for i in range(num_classes):
  weighted_f1 += np.sum(conf_arr, axis = 0)[i] * f1_score[i]

weighted_f1 = 100 * weighted_f1 / total
print('Weighted F1-score is : {} %'.format(weighted_f1))  